<a href="https://colab.research.google.com/github/nadkomash/malaria_recognition/blob/main/malara_recognition_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import tensorflow as tf
import tensorflow_datasets as tfds
from skimage.transform import resize



In [ ]:
malaria, info = tfds.load('malaria',split='train[:2500]',with_info=True)
fig = tfds.show_examples(malaria,info)
